The decorator is a callable that takes another function as argument. The decorator helps to transform the function as follows.

In [ ]:
@decorator
def target():
    print('running target()')

# has the same effect as

def target():
    print('running target')
    
target = decorator(target)

The first crucial fact about decorators is that they have the power to replace the decorated function with a different one. Second crucial fact is that they are exectuted immediately when a module is loaded.

In [2]:
registry = []

def register(func):
    print('running register (%s)'%func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('register -> ', registry)
    f1()
    f2()
    f3()


running register (<function f1 at 0x7f9564048050>)
running register (<function f2 at 0x7f95640b85f0>)


In [3]:

main()

running main()
register ->  [<function f1 at 0x7f9564048050>, <function f2 at 0x7f95640b85f0>]
running f1()
running f2()
running f3()


This emphasis the point that function decorators are executed as soon as the module is imported but the imported functions run only when they are explicitly called. This is what you call `importtime` and `runtime`

## Decorator-Enhanced Strategy Pattern

Decorators offer a way of implementing the `bestprom` functionality from the prev chapter. Here we can use decorators to register each promo code and that way minmise code reuse.

In [5]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)